# 라이브러리 import

In [365]:
import pandas as pd
import numpy as np
from numpy import arange
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc # 폰트
#한글폰트 적용-----------------------------
from matplotlib import font_manager, rc

plt.rc('font', family='AppleGothic')
#-----------------------------------------


from datetime import datetime, timedelta
import time
# datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')


import scipy.stats as stats
from scipy.stats import probplot
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import RobustScaler

# 데이터 import

In [366]:
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

## 개별 데이터 테이블 import

In [367]:
track=pd.read_csv('../data/track.csv')
track_curve=pd.read_csv('../data/track_curve.csv')
track_obstacle=pd.read_csv('../data/track_obstacle.csv')
track_road=pd.read_csv('../data/track_road.csv')
track_shortcut=pd.read_csv('../data/track_shortcut.csv')
track_straight=pd.read_csv('../data/track_straight.csv')
track_trigger=pd.read_csv('../data/track_trigger.csv')
match_indicator=pd.read_csv('../api/match-indicator-extraction.csv', encoding='cp949')
track_curve['all_section']=track_curve['hairpin']+track_curve['acute']+track_curve['obtuse']

## 테이블 변수 이름 수정

In [368]:
track_straight=pd.concat([track_straight.iloc[:, 0:1], track_straight.iloc[:, 1:].add_prefix('straight_')], axis=1)
track_curve=pd.concat([track_curve.iloc[:, 0:1], track_curve.iloc[:, 1:].add_prefix('curve_')], axis=1)
track_obstacle=pd.concat([track_obstacle.iloc[:, 0:1], track_obstacle.iloc[:, 1:].add_prefix('obstacle_')], axis=1)
track_trigger=pd.concat([track_trigger.iloc[:, 0:1], track_trigger.iloc[:, 1:].add_prefix('trigger_')], axis=1)
track_shortcut=pd.concat([track_shortcut.iloc[:, 0:1], track_shortcut.iloc[:, 1:].add_prefix('shortcut_')], axis=1)
track.rename(columns={'id':'track_id', 'name':'track_name'},inplace=True)
track['date']=pd.to_datetime(track['release_date'], errors='coerce')
track['year']=track['date'].dt.year.astype(int, errors = 'ignore')
track['month']=track['date'].dt.month.astype(int, errors = 'ignore')
track.drop(['date', 'release_date'], axis=1, inplace=True)
match_indicator=match_indicator.iloc[:, 1:]


## 데이터 merge

In [369]:
df_track=pd.merge(track, match_indicator, how='left', on='track_id')
df_track=pd.merge(df_track, track_straight, how='left', on='track_id')
df_track=pd.merge(df_track, track_trigger, how='left', on='track_id')
df_track=pd.merge(df_track, track_curve, how='left', on='track_id')
df_track=pd.merge(df_track, track_shortcut, how='left', on='track_id')
df_track=pd.merge(df_track, track_obstacle, how='left', on='track_id')
df_track['sum_straight_curve']=df_track['straight_all_section']+df_track['curve_all_section']
tmp1=df_track.sort_values(by='cnt_match', ascending=False)[:44]
tmp2=df_track.sort_values(by='cnt_match', ascending=False)[44:]
tmp1['track_upper']=1
tmp2['track_upper']=0
df_track=pd.concat([tmp1, tmp2])
df_track.sort_index(inplace=True)
df_track=pd.merge(df_track, track_road, how='left', on='track_id')
df_track.drop('track_id', axis=1, inplace=True)

## 파생변수로 이루어진 테이블 생성

In [370]:
track_Derived = df_track.copy()
# 직선, 커브, 지름길 관련 컬럼 제거 -> 파생변수로 새로 생성해줄 것임
track_Derived.drop(labels=['straight_uphill', 'straight_downhill', 'straight_all_section', 'sum_straight_curve',
                          'curve_all_section', 'sum_straight_curve', 'curve_hairpin', 'curve_acute', 'curve_obtuse',
                          'shortcut_right', 'shortcut_acute', 'shortcut_obtuse', 'shortcut_special',
                          'obstacle_fixed', 'obstacle_moved', 'curve_continuous_acute', 'curve_continuous', 'curve_continuous_obtuse','track_name'], axis=1, inplace=True)

track_Derived.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   difficulty       88 non-null     int64  
 1   lap              88 non-null     int64  
 2   length           88 non-null     float64
 3   direction        88 non-null     object 
 4   theme            88 non-null     object 
 5   year             88 non-null     int64  
 6   month            88 non-null     int64  
 7   cnt_match        88 non-null     int64  
 8   percent_retire   88 non-null     float64
 9   AVG_record       88 non-null     float64
 10  straight_jump    88 non-null     int64  
 11  trigger_accel    88 non-null     int64  
 12  trigger_warp     88 non-null     int64  
 13  trigger_decel    88 non-null     int64  
 14  trigger_jump     88 non-null     int64  
 15  track_upper      88 non-null     int64  
 16  fence_exist      88 non-null     int64  
 17  mean_road_level  8

In [371]:
# 오르막 내리막
track_Derived['updownhill'] = df_track['straight_uphill'] + df_track['straight_downhill']
# 직선구간 비율 (직선구간 / 전체구간)
track_Derived['ratio_straight'] = df_track['straight_all_section'] / df_track['sum_straight_curve']
# 곡선구간 비율 (곡선구간 / 전체구간)
track_Derived['ratio_curve'] = df_track['curve_all_section'] / df_track['sum_straight_curve']
# 헤어핀 비율
track_Derived['ratio_hairpin'] = df_track['curve_hairpin'] / df_track['curve_all_section']
# 예각 비율
track_Derived['ratio_acute'] = df_track['curve_acute'] / df_track['curve_all_section']
# 둔각 비율
track_Derived['ratio_obtuse'] = df_track['curve_obtuse'] / df_track['curve_all_section']
# 직각 지름길 비율
track_Derived['ratio_shortcut_right'] =  df_track['shortcut_right'] / (df_track['shortcut_right'] + df_track['shortcut_acute'] + df_track['shortcut_obtuse']+ df_track['shortcut_special'])
# 예각 지름길 비율
track_Derived['ratio_shortcut_acute'] =  df_track['shortcut_acute'] /  (df_track['shortcut_right'] + df_track['shortcut_acute'] + df_track['shortcut_obtuse']+ df_track['shortcut_special'])
# 둔각 지름길 비율
track_Derived['ratio_shortcut_obtuse'] =  df_track['shortcut_obtuse'] /  (df_track['shortcut_right'] + df_track['shortcut_acute'] + df_track['shortcut_obtuse']+ df_track['shortcut_special'])
# 특수 지름길 비율
track_Derived['ratio_shortcut_special'] =  df_track['shortcut_special'] /  (df_track['shortcut_right'] + df_track['shortcut_acute'] + df_track['shortcut_obtuse']+ df_track['shortcut_special'])
# 장애물 개수
track_Derived['obstacle'] = df_track['obstacle_fixed'] + df_track['obstacle_moved']
# 지름길 개수
track_Derived['shortcut'] = df_track['shortcut_right'] + df_track['shortcut_acute'] + df_track['shortcut_obtuse']+ df_track['shortcut_special']
track_Derived.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   difficulty              88 non-null     int64  
 1   lap                     88 non-null     int64  
 2   length                  88 non-null     float64
 3   direction               88 non-null     object 
 4   theme                   88 non-null     object 
 5   year                    88 non-null     int64  
 6   month                   88 non-null     int64  
 7   cnt_match               88 non-null     int64  
 8   percent_retire          88 non-null     float64
 9   AVG_record              88 non-null     float64
 10  straight_jump           88 non-null     int64  
 11  trigger_accel           88 non-null     int64  
 12  trigger_warp            88 non-null     int64  
 13  trigger_decel           88 non-null     int64  
 14  trigger_jump            88 non-null     int6

# 데이터 전처리

## 결측값 처리

In [400]:
# 파생변수
print(df_track.info())

print("*"*15 + '<<<null값>>>' +"*"*15)
print(df_track.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   track_name               88 non-null     object 
 1   difficulty               88 non-null     int64  
 2   lap                      88 non-null     int64  
 3   length                   88 non-null     float64
 4   direction                88 non-null     object 
 5   theme                    88 non-null     object 
 6   year                     88 non-null     int64  
 7   month                    88 non-null     int64  
 8   cnt_match                88 non-null     int64  
 9   percent_retire           88 non-null     float64
 10  AVG_record               88 non-null     float64
 11  straight_jump            88 non-null     int64  
 12  straight_uphill          88 non-null     int64  
 13  straight_downhill        88 non-null     int64  
 14  straight_all_section     88 

In [373]:
# 결측값 컬럼 확인
track_Derived[track_Derived.ratio_shortcut_acute.isnull()]

,difficulty,lap,length,direction,theme,year,month,cnt_match,percent_retire,AVG_record,straight_jump,trigger_accel,trigger_warp,trigger_decel,trigger_jump,track_upper,fence_exist,mean_road_level,updownhill,ratio_straight,ratio_curve,ratio_hairpin,ratio_acute,ratio_obtuse,ratio_shortcut_right,ratio_shortcut_acute,ratio_shortcut_obtuse,ratio_shortcut_special,obstacle,shortcut
0,2,1,3.5,단선형,해변,2013,8,7067,20.8078,55451.28545,0,0,0,0,0,1,0,1,9,0.444444,0.555556,0.900000,0.100000,0.000000,NaN,NaN,NaN,NaN,0,0
4,3,1,6.9,워프형,신화,2018,12,821,19.4186,101571.01640,0,5,1,8,0,1,0,1,7,0.666667,0.333333,0.238095,0.333333,0.428571,NaN,NaN,NaN,NaN,1,0
8,3,1,4.7,단선형,월드,2018,5,661,34.7700,82842.09884,0,0,1,0,0,1,1,2,17,0.481481,0.518519,0.642857,0.357143,0.000000,NaN,NaN,NaN,NaN,0,0
13,2,2,7.4,시계방향,빌리지,2004,6,2714,21.2592,116969.26630,0,0,0,0,0,1,0,2,14,0.653333,0.346667,0.307692,0.538462,0.153846,NaN,NaN,NaN,NaN,0,0
14,2,2,9.9,반시계방향,해변,2013,7,2140,27.2896,128865.35580,2,16,0,2,0,1,1,2,14,0.492958,0.507042,0.000000,0.611111,0.388889,NaN,NaN,NaN,NaN,24,0
15,2,2,6.4,시계방향,메이플,2022,7,1507,26.3288,98933.98068,0,0,0,0,2,1,0,1,12,0.688889,0.311111,0.142857,0.857143,0.000000,NaN,NaN,NaN,NaN,4,0
18,2,2,8.7,반시계방향,WKC,2011,6,1311,20.3944,120722.01380,0,0,0,0,0,1,0,1,10,0.648148,0.351852,0.157895,0.315789,0.526316,NaN,NaN,NaN,NaN,6,0
21,2,2,8.6,시계방향,차이나,2014,6,1039,26.6023,124708.21470,0,0,0,0,0,1,0,1,22,0.586957,0.413043,0.210526,0.421053,0.368421,NaN,NaN,NaN,NaN,0,0
23,3,2,9.4,반시계방향,빌리지,2010,9,894,34.4339,145489.06010,0,0,0,4,0,1,0,1,24,0.712000,0.288000,0.444444,0.333333,0.222222,NaN,NaN,NaN,NaN,8,0
24,3,2,9.1,반시계방향,노르테유,2022,6,852,40.1849,143369.54010,2,16,0,2,2,1,1,1,12,0.571429,0.428571,0.190476,0.428571,0.380952,NaN,NaN,NaN,NaN,2,0


- 파생변수에서 결측값 확인, 결측값은 일부 지름길 / 지름길 전체 = 0/0 의 결과에 따라 NaN으로 나오는 것을 확인

In [374]:
# 결측값 0 채워주기  (0%를 의미)

track_Derived = track_Derived.fillna(0)
track_Derived.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   difficulty              88 non-null     int64  
 1   lap                     88 non-null     int64  
 2   length                  88 non-null     float64
 3   direction               88 non-null     object 
 4   theme                   88 non-null     object 
 5   year                    88 non-null     int64  
 6   month                   88 non-null     int64  
 7   cnt_match               88 non-null     int64  
 8   percent_retire          88 non-null     float64
 9   AVG_record              88 non-null     float64
 10  straight_jump           88 non-null     int64  
 11  trigger_accel           88 non-null     int64  
 12  trigger_warp            88 non-null     int64  
 13  trigger_decel           88 non-null     int64  
 14  trigger_jump            88 non-null     int6

## 사분위수 & 왜도 확인

In [375]:
track_Derived.describe().T

,count,mean,std,min,25%,50%,75%,max
difficulty,88.0,2.590909,1.120718,1.000000,2.000000,2.000000,3.000000,6.000000
lap,88.0,2.261364,0.702938,1.000000,2.000000,2.000000,3.000000,3.000000
length,88.0,7.296591,2.293341,1.500000,6.000000,6.900000,8.425000,17.100000
year,88.0,2012.500000,5.785436,2004.000000,2007.750000,2012.500000,2017.250000,2022.000000
month,88.0,7.795455,3.107866,1.000000,6.000000,7.000000,11.000000,12.000000
cnt_match,88.0,753.215909,1500.935467,29.000000,40.750000,275.000000,825.750000,10325.000000
percent_retire,88.0,27.586106,12.823972,0.000000,19.943875,27.093800,34.486850,65.104700
AVG_record,88.0,122979.385492,49936.491163,51731.585020,95568.672265,113619.425000,135899.756675,373421.855600
straight_jump,88.0,0.409091,1.035423,0.000000,0.000000,0.000000,0.000000,4.000000
trigger_accel,88.0,1.977273,5.521072,0.000000,0.000000,0.000000,0.000000,39.000000


대체적으로 변수들이 75%와 max에 차이가 어느정도 있음을 확인할 수 있다. -> 왼쪽으로 치우친 분포로 예상이 됨

In [376]:
# 수치형 변수, 범주형 변수 나눠줌

numerical_feats = track_Derived.dtypes[track_Derived.dtypes != "object"].index 
print("Number of Numerical features: ", len(numerical_feats)) 
categorical_feats = track_Derived.dtypes[track_Derived.dtypes == "object"].index 
print("Number of Categorical features: ", len(categorical_feats))

Number of Numerical features:  28
Number of Categorical features:  2


In [377]:
for col in numerical_feats:
    print('{:15}'.format(col), 
          'Skewness: {:05.2f}'.format(track_Derived[col].skew()) , 
          '   ' ,
          'Kurtosis: {:06.2f}'.format(track_Derived[col].kurt())  
         )

difficulty      Skewness: 00.97     Kurtosis: 000.90
lap             Skewness: -0.42     Kurtosis: -00.90
length          Skewness: 01.59     Kurtosis: 005.45
year            Skewness: 00.06     Kurtosis: -01.15
month           Skewness: -0.10     Kurtosis: -00.93
cnt_match       Skewness: 04.39     Kurtosis: 022.61
percent_retire  Skewness: 00.34     Kurtosis: 000.76
AVG_record      Skewness: 02.57     Kurtosis: 009.47
straight_jump   Skewness: 02.41     Kurtosis: 004.60
trigger_accel   Skewness: 04.42     Kurtosis: 024.18
trigger_warp    Skewness: 02.87     Kurtosis: 010.56
trigger_decel   Skewness: 01.21     Kurtosis: 001.14
trigger_jump    Skewness: 02.71     Kurtosis: 007.03
track_upper     Skewness: 00.00     Kurtosis: -02.05
fence_exist     Skewness: 00.33     Kurtosis: -01.94
mean_road_level Skewness: 01.11     Kurtosis: 000.17
updownhill      Skewness: 01.69     Kurtosis: 005.74
ratio_straight  Skewness: -0.47     Kurtosis: -00.93
ratio_curve     Skewness: 00.47     Kurtosis: 

- 대체적으로 왜도가 양수이며, 우측으로 꼬리를 가지고 있음을 유추가능 
- 정규성을 높이고 더 좋은 모델 성능을 얻기 위해 일부 변수에 log를 진행하기로 결정

## 로그 변환 진행해주기

In [378]:
df_track_Derived = track_Derived.copy()

In [379]:
# Log 변환 진행해줄 친구들
track_skew_list = [ 'AVG_record', 'straight_jump', 'trigger_accel']

for i in range(len(track_skew_list)):
    skew_col = track_skew_list[i]
    df_track_Derived[skew_col] = np.log1p(df_track_Derived[skew_col])
    df_track_Derived.rename(columns = {skew_col:'log_'+ skew_col}, inplace = True )

In [380]:
df_track_Derived_log = df_track_Derived.copy()

print("log_AVG_record skew: ", df_track_Derived_scale['log_AVG_record'].skew())
print("log_straight_jump skew: ", df_track_Derived_scale['log_straight_jump'].skew())
print("log_trigger_accel skew: ", df_track_Derived_scale['log_trigger_accel'].skew())

log_AVG_record skew:  0.6295761409743453
log_straight_jump skew:  2.1455255092618573
log_trigger_accel skew:  1.8750467296640743


전보다 skew값이 완화 된 것을 볼 수 있음

In [381]:
# 테마 변수 재그룹화
df_track_Derived.replace( {'해변' : '자연',
                        '포레스트' : '자연', 
                        '사막' : '자연', 
                        '아이스' : '자연', 
                        '빌리지' : '자연', 
                        '코리아' : '세계',
                        '월드' : '세계', 
                        '차이나' : '세계', 
                        '도검' : '세계', 
                        '신화' : '고대', 
                        '올림포스' : '고대', 
                        '님프' : '고대', 
                        '어비스' : '고대', 
                        '쥐라기' : '고대', 
                        '네모' : '동화', 
                        '놀이동산' : '동화',
                        '대저택' : '동화', 
                        '공동묘지' : '다크', 
                        '광산' : '다크', 
                        '문힐시티' : '다크',
                        '메이플' : '콜라보', 
                        '마비노기' : '콜라보', 
                        'WKC' : '콜라보', 
                        '해적' : '과거여행', 
                        '1920' : '과거여행', 
                        '노르테유' : '공장',
                        '브로디' : '공장'}, inplace = True)

In [382]:
feature_names = list(df_track_Derived.select_dtypes(exclude='object').columns)

# 범주형 처리해야 하는 변수 제거, 비연속형 변수 제거
# 독립형 스케일링 필요한 변수들
feature_names.remove("track_upper") 
feature_names.remove("fence_exist") # 범주형
feature_names.remove("cnt_match") # 종속

# feature_names = ["scale({})".format(name) for name in feature_names] 
feature_names

['difficulty',
 'lap',
 'length',
 'year',
 'month',
 'percent_retire',
 'log_AVG_record',
 'log_straight_jump',
 'log_trigger_accel',
 'trigger_warp',
 'trigger_decel',
 'trigger_jump',
 'mean_road_level',
 'updownhill',
 'ratio_straight',
 'ratio_curve',
 'ratio_hairpin',
 'ratio_acute',
 'ratio_obtuse',
 'ratio_shortcut_right',
 'ratio_shortcut_acute',
 'ratio_shortcut_obtuse',
 'ratio_shortcut_special',
 'obstacle',
 'shortcut']

# 회귀분석 진행하기

## 상관관계 분석

In [383]:
corr_df = df_track_Derived.drop(labels='track_upper',axis=1)

In [384]:
# 종속변수와 상관계수 확인해보기
corr_df.select_dtypes(exclude='object').corr().T.loc[:,['cnt_match']]

,cnt_match
difficulty,-0.235999
lap,-0.107581
length,-0.250704
year,-0.114287
month,-0.054501
cnt_match,1.000000
percent_retire,-0.128021
log_AVG_record,-0.470101
log_straight_jump,0.040693
log_trigger_accel,0.042874


## model1 - 상관관계 높은 일부 변수로 회귀분석 진행

변수가 많으므로 단계적 회귀분석을 선택, 전부 유의한 회귀분석을 찾고 그 이후, 다른 변수들을 추가 제거하며 모형 적합 진행

In [385]:
model = ols('cnt_match ~ log_AVG_record + C(theme)' 
            , data = df_track_Derived).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.302
Model:                            OLS   Adj. R-squared:                  0.231
Method:                 Least Squares   F-statistic:                     4.269
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           0.000267
Time:                        23:35:41   Log-Likelihood:                -752.17
No. Observations:                  88   AIC:                             1522.
Df Residuals:                      79   BIC:                             1545.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept         2.685e+04   5378.403      4.992      0.000    1.61e+04    3.76e+04
C(theme)[T.공장]     494.4480    658.547      0.751      0.455    -816.357    1805.253
C(theme)[T.과거여행]   987.4321    779.815      1.266      0.209    -564.751    2539.615
C(theme)[T.다크]     184.1241    646.422      0.285      0.777   -1102.547    1470.795
C(theme)[T.동화]    -109.5481    681.330     -0.161      0.873   -1465.701    1246.605
C(theme)[T.세계]     226.8784    572.113      0.397      0.693    -911.884    1365.641
C(theme)[T.자연]    1071.5701    540.363      1.983      0.051      -3.996    2147.137
C(theme)[T.콜라보]    221.1485    601.961      0.367      0.714    -977.025    1419.322
log_AVG_record   -2278.0990    460.212     -4.950      0.000   -3194.129   -1362.070
==============================================================================
Omnibus:                       75.575   Durbin-Watson:                   0.950
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              586.138
Skew:                           2.665   Prob(JB):                    5.27e-128
Kurtosis:                      14.465   Cond. No.                         451.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

theme에서 자연 테마만 유의하다고 나오므로, 해당 테마의 값만 가지는 범주로 다시 묶어주기로 진행

In [386]:
df_track_Derived['theme'].replace({'자연' : 1,
                                      '세계' : 0,
                                      '고대' : 0,
                                      '동화' : 0,
                                      '다크' : 0,
                                      '콜라보' : 0,
                                      '과거여행' : 0,
                                      '공장' : 0}, inplace = True )
df_track_Derived.rename(columns = {'theme' : 'nature_theme'}, inplace=True)

In [387]:
model1 = ols('cnt_match ~ log_AVG_record + C(nature_theme)' 
            , data = df_track_Derived).fit()

model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.279
Model:                            OLS   Adj. R-squared:                  0.262
Method:                 Least Squares   F-statistic:                     16.48
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           8.93e-07
Time:                        23:36:06   Log-Likelihood:                -753.56
No. Observations:                  88   AIC:                             1513.
Df Residuals:                      85   BIC:                             1521.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             2.518e+04   4770.154      5.279      0.000    1.57e+04    3.47e+04
C(nature_theme)[T.1]   821.4046    312.816      2.626      0.010     199.443    1443.367
log_AVG_record       -2113.6879    409.098     -5.167      0.000   -2927.085   -1300.291
==============================================================================
Omnibus:                       77.634   Durbin-Watson:                   0.918
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              619.037
Skew:                           2.756   Prob(JB):                    3.78e-135
Kurtosis:                      14.767   Cond. No.                         408.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

👉🏻 모든 변수가 유의수준 5%하에서 유의하므로 해당 모형을 기준으로 단계선택법 진행

## model2 - difficulty 변수 추가

In [388]:
mode2 = ols('cnt_match ~ log_AVG_record + difficulty + C(nature_theme)' 
            , data = df_track_Derived).fit()

mode2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     13.95
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           1.83e-07
Time:                        23:36:13   Log-Likelihood:                -750.19
No. Observations:                  88   AIC:                             1508.
Df Residuals:                      84   BIC:                             1518.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             3.665e+04   6402.267      5.724      0.000    2.39e+04    4.94e+04
C(nature_theme)[T.1]  1102.6932    321.780      3.427      0.001     462.798    1742.589
log_AVG_record       -3206.6543    579.166     -5.537      0.000   -4358.390   -2054.919
difficulty             463.1283    179.061      2.586      0.011     107.046     819.211
==============================================================================
Omnibus:                       70.635   Durbin-Watson:                   1.089
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              493.391
Skew:                           2.474   Prob(JB):                    7.27e-108
Kurtosis:                      13.491   Cond. No.                         580.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## model3 - length 변수 추가

In [389]:
model3 = ols('cnt_match ~ log_AVG_record + difficulty + length + C(nature_theme)' 
            , data = df_track_Derived).fit()

model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.434
Method:                 Least Squares   F-statistic:                     17.65
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           1.63e-10
Time:                        23:36:26   Log-Likelihood:                -740.90
No. Observations:                  88   AIC:                             1492.
Df Residuals:                      83   BIC:                             1504.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             6.496e+04   8642.882      7.516      0.000    4.78e+04    8.22e+04
C(nature_theme)[T.1]  1358.5259    297.002      4.574      0.000     767.801    1949.251
log_AVG_record       -5932.8686    809.948     -7.325      0.000   -7543.823   -4321.914
difficulty             469.5574    162.104      2.897      0.005     147.138     791.977
length                 464.0888    105.093      4.416      0.000     255.062     673.115
==============================================================================
Omnibus:                       56.252   Durbin-Watson:                   1.363
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              261.321
Skew:                           2.009   Prob(JB):                     1.80e-57
Kurtosis:                      10.424   Cond. No.                     1.02e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

다중공선성 의심되므로 다중공선성 확인 진행함

### 다중공선성 확인

In [390]:
x = df_track_Derived.loc[:,['log_AVG_record','difficulty','length']]

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_vif(x):
  vif = pd.DataFrame()
  vif['vif_factor'] = [variance_inflation_factor(x.values, i)
                      for i in range(x.shape[1])]
  vif['feature'] = x.columns
  return vif
vif = feature_vif(x)
print(vif)                         

   vif_factor         feature
0   13.500616  log_AVG_record
1   10.635789      difficulty
2   19.974035          length


In [391]:
x = df_track_Derived.loc[:,['log_AVG_record','difficulty']]

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

def feature_vif(x):
  vif = pd.DataFrame()
  vif['vif_factor'] = [variance_inflation_factor(x.values, i)
                      for i in range(x.shape[1])]
  vif['feature'] = x.columns
  return vif
vif = feature_vif(x)
print(vif)                         

   vif_factor         feature
0    7.042935  log_AVG_record
1    7.042935      difficulty


length 변수 다중공선성 문제로 추가 안함

## model4 - trigger_decel 추가

In [392]:
model4 = ols('cnt_match ~ log_AVG_record + difficulty + trigger_decel + C(nature_theme)' 
            , data = df_track_Derived).fit()

model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     10.73
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           4.66e-07
Time:                        23:36:44   Log-Likelihood:                -749.64
No. Observations:                  88   AIC:                             1509.
Df Residuals:                      83   BIC:                             1522.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             3.623e+04   6413.854      5.649      0.000    2.35e+04     4.9e+04
C(nature_theme)[T.1]  1113.8653    321.897      3.460      0.001     473.624    1754.106
log_AVG_record       -3168.4104    580.257     -5.460      0.000   -4322.519   -2014.302
difficulty             495.1593    181.765      2.724      0.008     133.635     856.684
trigger_decel          -43.6362     42.859     -1.018      0.312    -128.882      41.609
==============================================================================
Omnibus:                       69.738   Durbin-Watson:                   1.133
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              486.861
Skew:                           2.430   Prob(JB):                    1.90e-106
Kurtosis:                      13.448   Cond. No.                         597.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## model5 - ratio_hairpin 추가

In [393]:
model5 = ols('cnt_match ~ log_AVG_record + difficulty + ratio_hairpin + C(nature_theme)' 
            , data = df_track_Derived).fit()

model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.307
Method:                 Least Squares   F-statistic:                     10.63
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           5.32e-07
Time:                        23:36:52   Log-Likelihood:                -749.79
No. Observations:                  88   AIC:                             1510.
Df Residuals:                      83   BIC:                             1522.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             3.547e+04   6552.906      5.413      0.000    2.24e+04    4.85e+04
C(nature_theme)[T.1]  1028.6755    333.250      3.087      0.003     365.855    1691.495
log_AVG_record       -3102.3058    592.231     -5.238      0.000   -4280.229   -1924.383
difficulty             411.6173    188.811      2.180      0.032      36.081     787.154
ratio_hairpin          652.8559    749.213      0.871      0.386    -837.299    2143.011
==============================================================================
Omnibus:                       74.744   Durbin-Watson:                   1.083
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              600.110
Skew:                           2.601   Prob(JB):                    4.87e-131
Kurtosis:                      14.688   Cond. No.                         593.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## model6 - trigger_warp 추가

In [394]:
model6 = ols('cnt_match ~ log_AVG_record + difficulty + log_straight_jump + trigger_warp + C(nature_theme)' 
            , data = df_track_Derived).fit()

model6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.366
Model:                            OLS   Adj. R-squared:                  0.327
Method:                 Least Squares   F-statistic:                     9.456
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           3.80e-07
Time:                        23:37:01   Log-Likelihood:                -747.95
No. Observations:                  88   AIC:                             1508.
Df Residuals:                      82   BIC:                             1523.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              3.87e+04   6407.151      6.040      0.000     2.6e+04    5.14e+04
C(nature_theme)[T.1]  1095.3038    325.483      3.365      0.001     447.814    1742.793
log_AVG_record       -3386.1455    579.685     -5.841      0.000   -4539.324   -2232.967
difficulty             451.7489    176.761      2.556      0.012     100.114     803.383
log_straight_jump      563.2547    293.161      1.921      0.058     -19.937    1146.446
trigger_warp           -48.2295     99.714     -0.484      0.630    -246.593     150.134
==============================================================================
Omnibus:                       70.914   Durbin-Watson:                   1.189
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              525.894
Skew:                           2.454   Prob(JB):                    6.36e-115
Kurtosis:                      13.924   Cond. No.                         589.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## model7 - ratio_hairpin 추가

In [395]:
model7 = ols('cnt_match ~ log_AVG_record + difficulty + log_straight_jump + ratio_hairpin + C(nature_theme)' 
            , data = df_track_Derived).fit()

model7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.371
Model:                            OLS   Adj. R-squared:                  0.333
Method:                 Least Squares   F-statistic:                     9.673
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           2.75e-07
Time:                        23:37:09   Log-Likelihood:                -747.58
No. Observations:                  88   AIC:                             1507.
Df Residuals:                      82   BIC:                             1522.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             3.754e+04   6507.940      5.768      0.000    2.46e+04    5.05e+04
C(nature_theme)[T.1]  1049.7629    327.136      3.209      0.002     398.986    1700.540
log_AVG_record       -3287.4869    588.035     -5.591      0.000   -4457.277   -2117.697
difficulty             396.6020    185.399      2.139      0.035      27.784     765.421
log_straight_jump      593.5436    289.055      2.053      0.043      18.521    1168.566
ratio_hairpin          706.1240    735.563      0.960      0.340    -757.145    2169.393
==============================================================================
Omnibus:                       75.682   Durbin-Watson:                   1.199
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              652.897
Skew:                           2.607   Prob(JB):                    1.68e-142
Kurtosis:                      15.283   Cond. No.                         600.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## model8 - ratio_obtuse 추가

In [396]:
model8 = ols('cnt_match ~ log_AVG_record + difficulty + log_straight_jump + ratio_obtuse + C(nature_theme)' 
            , data = df_track_Derived).fit()

model8.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     9.395
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           4.16e-07
Time:                        23:37:20   Log-Likelihood:                -748.05
No. Observations:                  88   AIC:                             1508.
Df Residuals:                      82   BIC:                             1523.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             3.857e+04   6497.599      5.936      0.000    2.56e+04    5.15e+04
C(nature_theme)[T.1]  1138.8004    321.739      3.540      0.001     498.759    1778.841
log_AVG_record       -3375.9426    589.871     -5.723      0.000   -4549.385   -2202.500
difficulty             450.2893    177.320      2.539      0.013      97.544     803.034
log_straight_jump      572.9536    295.575      1.938      0.056     -15.038    1160.946
ratio_obtuse          -132.8641    675.531     -0.197      0.845   -1476.711    1210.983
==============================================================================
Omnibus:                       71.480   Durbin-Watson:                   1.202
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              536.169
Skew:                           2.475   Prob(JB):                    3.74e-117
Kurtosis:                      14.032   Cond. No.                         596.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## model7 - mean_road_level 추가

In [397]:
model7 = ols('cnt_match ~ log_AVG_record + difficulty + log_straight_jump + mean_road_level + C(nature_theme)' 
            , data = df_track_Derived).fit()

model7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.326
Method:                 Least Squares   F-statistic:                     9.417
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           4.03e-07
Time:                        23:37:44   Log-Likelihood:                -748.02
No. Observations:                  88   AIC:                             1508.
Df Residuals:                      82   BIC:                             1523.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             3.877e+04   6409.885      6.048      0.000     2.6e+04    5.15e+04
C(nature_theme)[T.1]  1135.5609    318.566      3.565      0.001     501.832    1769.290
log_AVG_record       -3385.7086    580.719     -5.830      0.000   -4540.944   -2230.474
difficulty             446.9726    177.681      2.516      0.014      93.509     800.436
log_straight_jump      587.3171    290.503      2.022      0.046       9.415    1165.219
mean_road_level        -83.5665    253.497     -0.330      0.743    -587.852     420.719
==============================================================================
Omnibus:                       70.018   Durbin-Watson:                   1.186
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              504.228
Skew:                           2.426   Prob(JB):                    3.22e-110
Kurtosis:                      13.676   Cond. No.                         592.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 최종모델 

In [349]:
model = ols('cnt_match ~ log_AVG_record + difficulty + C(nature_theme)' 
            , data = df_track_Derived).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.309
Method:                 Least Squares   F-statistic:                     13.95
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           1.83e-07
Time:                        23:28:49   Log-Likelihood:                -750.19
No. Observations:                  88   AIC:                             1508.
Df Residuals:                      84   BIC:                             1518.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept             3.665e+04   6402.267      5.724      0.000    2.39e+04    4.94e+04
C(nature_theme)[T.1]  1102.6932    321.780      3.427      0.001     462.798    1742.589
log_AVG_record       -3206.6543    579.166     -5.537      0.000   -4358.390   -2054.919
difficulty             463.1283    179.061      2.586      0.011     107.046     819.211
==============================================================================
Omnibus:                       70.635   Durbin-Watson:                   1.089
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              493.391
Skew:                           2.474   Prob(JB):                    7.27e-108
Kurtosis:                      13.491   Cond. No.                         580.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## log_AVG_record 단일 변수 설명력

In [399]:
model = ols('cnt_match ~ log_AVG_record'
            , data = df_track_Derived).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.221
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     24.40
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           3.82e-06
Time:                        23:38:39   Log-Likelihood:                -756.99
No. Observations:                  88   AIC:                             1518.
Df Residuals:                      86   BIC:                             1523.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        2.51e+04   4930.827      5.090      0.000    1.53e+04    3.49e+04
log_AVG_record -2088.1987    422.768     -4.939      0.000   -2928.634   -1247.764
==============================================================================
Omnibus:                       87.803   Durbin-Watson:                   0.959
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              908.559
Skew:                           3.145   Prob(JB):                    5.12e-198
Kurtosis:                      17.430   Cond. No.                         408.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [403]:
model = ols('cnt_match ~ log_AVG_record + difficulty + C(nature_theme) + C(difficulty)' 
            , data = df_track_Derived).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              cnt_match   R-squared:                       0.364
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     6.532
Date:                Sat, 26 Nov 2022   Prob (F-statistic):           4.22e-06
Time:                        23:55:14   Log-Likelihood:                -748.09
No. Observations:                  88   AIC:                             1512.
Df Residuals:                      80   BIC:                             1532.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              3.96e+04   6579.668      6.018      0.000    2.65e+04    5.27e+04
C(nature_theme)[T.1]  1107.9463    355.051      3.121      0.003     401.373    1814.519
C(difficulty)[T.2]    -898.1933    422.887     -2.124      0.037   -1739.765     -56.622
C(difficulty)[T.3]   -1839.9696    494.508     -3.721      0.000   -2824.073    -855.866
C(difficulty)[T.4]   -2731.9250    620.659     -4.402      0.000   -3967.076   -1496.774
C(difficulty)[T.5]   -3372.0556    767.597     -4.393      0.000   -4899.622   -1844.490
C(difficulty)[T.6]   -2409.1793    708.722     -3.399      0.001   -3819.581    -998.777
log_AVG_record       -3523.9985    601.559     -5.858      0.000   -4721.138   -2326.859
difficulty            1288.3626    293.450      4.390      0.000     704.379    1872.346
==============================================================================
Omnibus:                       63.377   Durbin-Watson:                   1.183
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              386.568
Skew:                           2.195   Prob(JB):                     1.14e-84
Kurtosis:                      12.282   Cond. No.                     5.26e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.59e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""